In [0]:
from pyspark.sql import SparkSession, functions as F
from delta.tables import DeltaTable
spark = SparkSession.builder.appName("Option_2_level_2").getOrCreate()

In [0]:
temp_silver_df = spark.read.table("delta_silver_valid")
delta_facts_sales_object = DeltaTable.forName(spark, "delta_facts_sales")

if not spark.catalog.tableExists("delta_facts_sales"):
    
    print("Table not exists. Creating one...")
    (
        temp_silver_df
        .withColumn("ingestion_timestamp", F.current_timestamp())
        .withColumn("last_updated_timestamp", F.lit(None).cast("timestamp"))
        .write
        .format("delta")
        .mode("overwrite")
        .saveAsTable("delta_facts_sales")
    )

    print("Number of records in the table: ", temp_silver_df.count())

else:
    print("Table exists. processing updated and new records..")

    change_condition = """
        NOT (t.Shipdate <=> s.Shipdate) OR 
        NOT (t.Amount <=> s.Amount) OR 
        NOT (t.Boxes <=> s.Boxes)
    """

    all_columns = temp_silver_df.columns
    base_map = {c: F.col(f"s.{c}") for c in all_columns}
    
    update_map = base_map.copy()
    update_map['last_updated_timestamp'] = F.current_timestamp()

    insert_map = base_map.copy()
    insert_map['ingestion_timestamp'] = F.current_timestamp()

    delta_merge = (
        delta_facts_sales_object.alias("t")
        .merge(temp_silver_df.alias("s"), "t.ShipmentID = s.ShipmentID")
        .whenMatchedUpdate(condition=change_condition, set=update_map)
        .whenNotMatchedInsert(values=insert_map)
        .execute()
    )

    updated_data = spark.read.table("delta_facts_sales")
    print("Number of records in the table: ", updated_data.count())

    latest_history = delta_facts_sales_object.history(1).collect()[0]
    metrics = latest_history["operationMetrics"]
    rows_updated = int(metrics.get("numTargetRowsUpdated", 0))
    rows_inserted = int(metrics.get("numTargetRowsInserted", 0))

    print("Rows updated: ", rows_updated)
    print("Rows inserted: ", rows_inserted) 





In [0]:
%sql
show tables;